In [1]:
import pyart
import matplotlib.pyplot as plt
import numpy as np
import glob
import pandas as pd
import xarray as xr


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
# Helper function used for visualization in the following examples
def identify_axes(ax_dict, fontsize=48):
    """
    Helper to identify the Axes in the examples below.

    Draws the label in a large font in the center of the Axes.

    Parameters
    ----------
    ax_dict : dict[str, Axes]
        Mapping between the title / label and the Axes.
    fontsize : int, optional
        How big the label should be.
    """
    kw = dict(ha="center", va="center", fontsize=fontsize, color="darkgrey")
    for k, ax in ax_dict.items():
        ax.text(0.5, 0.5, k, transform=ax.transAxes, **kw)

In [3]:
def subset_points(file, lats, lons, sites):
    """Subset a radar file for a set of latitudes and longitudes"""
    
    # Read in the file
    radar = pyart.io.read(file)
    print("hey", radar.altitude['data'][0])
    column_list = []
    for lat, lon in zip(lats, lons):
        print(lat, lon)
        # Make sure we are interpolating from the radar's location above sea level
        #da = pyart.util.columnsect.get_field_location(radar, lat, lon).interp(height=np.arange(radar.altitude['data'][0], 10100, 100))
        da = pyart.util.columnsect.get_field_location(radar, lat, lon)
        # Add the latitude and longitude of the extracted column
        da["latitude"], da["longitude"] = lat, lon
        # Time is based off the start of the radar volume
        #dt = pd.to_datetime(radar.time["data"], unit='s')[-1]
        #da["time"] = [dt]
        # Make sure there are no duplicate height indices
        index = np.unique(da['height'], return_index=True)
        da = da.isel(height=index[1])
        # Append to the xrray dataset list. Interpolate heightsI'm so 
        column_list.append(da.interp(height=np.arange(100, 3000, 100)))
    # Concatenate the extracted radar columns for this scan across all sites    
    ds = xr.concat(column_list, dim='site')
    ds["site"] = sites
    # Add attributes for Time, Latitude, Longitude, and Sites
    #ds.time.attrs.update(long_name=('Time in Seconds that Cooresponds to the Start'
    #                                + " of each Individual Radar Volume Scan before"
    #                                + " Concatenation"),
    #                     units='seconds',
    #                     description=('Time in Seconds that Cooresponds to the Minimum'
    #                                  + ' Height Gate'))
    #ds.site.attrs.update(long_name="SAIL/SPLASH In-Situ Ground Observation Site Identifers")
    #ds.latitude.attrs.update(long_name='Latitude of SAIL Ground Observation Site',
    #                         units='Degrees North')
    #ds.longitude.attrs.update(long_name='Longitude of SAIL Ground Observation Site',
    #                         units='Degrees East')
    return ds

In [4]:
def match_datasets(column, ground, site, DataSet=False):
    """
    Time synchronization of a Ground Instrumentation Dataset to 
    a Radar Column for Specific Locations
    
    Parameters
    ----------
    column : Xarray DataSet
             Xarray DataSet containing the extracted radar column above multiple locations.
             Dimensions should include Time, Height, Site
             
    ground : str 
             String containing the path of the ground instrumentation file that is desired
             to be included within the extracted radar column dataset
             
    site : str
           Location of the ground instrument. Should be included within the filename. 
             
    Returns
    -------
    ds : Xarray DataSet
         Xarray Dataset containing the time-synced in-situ ground observations with
         the inputed radar column 
    """
    # Check to see if input is xarray DataSet or a file path
    if DataSet == True:
        grd_ds = ground
    else:
        # Read in the file
        grd_ds = xr.open_dataset(ground)

    # Resample the ground data to 5 min and interpolate to the CSU X-Band time. 
    # Keep data variable attributes to help distingish between instruments/locations
    matched = grd_ds.resample(time='5Min', closed='right').sum(keep_attrs=True).interp(time=column.time, method='linear') 
    
    # Add SAIL site location as a dimension for the Pluvio data
    matched = matched.assign_coords(coords=dict(site=site))
    matched = matched.expand_dims('site')
    
    # Remove Lat/Lon Data variables as it is included within the Matched Dataset with Site Identfiers
    if 'lat' in matched.data_vars:
        del matched['lat']
    if 'lon' in matched.data_vars:
        del matched['lon']
        
    # Update the individual Variables to Hold Global Attributes
    # global attributes will be lost on merging into the matched dataset.
    # Need to keep as many references and descriptors as possible
    for var in matched.data_vars:
        matched[var].attrs.update(platform_id=matched.platform_id)
        matched[var].attrs.update(data_level=matched.data_level)
        matched[var].attrs.update(location_description=matched.location_description)
        matched[var].attrs.update(datastream=matched.datastream)
        matched[var].attrs.update(doi=matched.doi)
        
    # Merge the two DataSets
    column = xr.merge([column, matched])
    
    # Tranpose the xarray DataSet to match the order of dimensions across all variables
    ##column = column.transpose("time", "height", "site")
    
    return column 

## Grab all the Data (Locally Stored)

In [5]:
file_list = sorted(glob.glob("/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/*"))
file_list[:10]

['/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_000056_V06',
 '/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_000540_V06',
 '/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_001029_V06',
 '/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_001514_V06',
 '/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_002105_V06',
 '/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_002548_V06',
 '/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_003037_V06',
 '/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_003510_V06',
 '/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_003938_V06',
 '/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_004407_V06']

In [6]:
ld_files = sorted(glob.glob("/Users/celsloaner/ARM/Observations/LD/TRACER/M1"))

In [7]:
file_list[0][-3:]

'V06'

In [8]:
radar = pyart.io.read(file_list[10])

In [9]:
da = pyart.util.columnsect.get_field_location(radar, 29.62, -95.059)

In [10]:
da

<xarray.Dataset>
Dimensions:                       (height: 12)
Coordinates:
  * height                        (height) float64 175.0 175.0 ... 1.867e+03
Data variables:
    velocity                      (height) float64 nan nan nan ... nan nan nan
    differential_reflectivity     (height) float64 -10.47 nan 4.031 ... nan nan
    cross_correlation_ratio       (height) float64 0.845 nan 0.6217 ... nan nan
    spectrum_width                (height) float64 nan nan nan ... nan nan nan
    reflectivity                  (height) float64 8.5 8.5 -10.0 ... nan nan nan
    clutter_filter_power_removed  (height) float64 nan nan 46.0 ... 11.0 10.0
    differential_phase            (height) float64 321.6 nan 326.2 ... nan nan
    base_time                     datetime64[ns] 2022-08-10T00:48:41
    time_offset                   (height) datetime64[ns] 2022-08-10T00:48:58...
Attributes:
    distance_from_radar:    16.578 km
    azimuth:                6.608 degrees
    latitude_of_location:   29.62 degrees
    longitude_of_location:  -95.059 degrees

## Extract the Radar Column above the TRACER M1 site. 

In [11]:
%%time
lats = [29.62]
lons = [-95.059]
sites = ["M1"]

ds_list = []
for file in file_list[:]:
    if file[-3:] != "MDM":
        print(file)
        ds_list.append(subset_points(file, lats, lons, sites))

/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_000056_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_000540_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_001029_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_001514_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_002105_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_002548_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_003037_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_003510_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_003938_V06
hey 34.0
29.62 -95.059
/Users/cel

hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_061411_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_062124_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_062837_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_063550_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_064303_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_065017_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_065729_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_070442_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_071155_V06
hey 34.0
2

hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_131453_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_132008_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_132538_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_133033_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_133534_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_134034_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_134741_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_135454_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_140207_V06
hey 34.0
2

hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_190335_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_190806_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_191251_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_191736_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_192222_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_192658_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_193129_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_193604_V06
hey 34.0
29.62 -95.059
/Users/celsloaner/ARM/Observations/NEXRAD/KHGX/2022/08/10/KHGX/KHGX20220810_194044_V06
hey 34.0
2

In [12]:
ds = xr.concat(ds_list, dim='height')

In [14]:
ds

<xarray.Dataset>
Dimensions:                       (height: 8323, site: 1)
Coordinates:
  * height                        (height) int64 100 200 300 ... 2700 2800 2900
  * site                          (site) <U2 'M1'
Data variables:
    velocity                      (site, height) float64 nan nan ... -1.631
    differential_reflectivity     (site, height) float64 nan nan ... 0.04926
    cross_correlation_ratio       (site, height) float64 nan nan ... 0.998
    spectrum_width                (site, height) float64 nan nan nan ... 2.5 2.5
    reflectivity                  (site, height) float64 nan nan ... 36.16 37.55
    clutter_filter_power_removed  (site, height) float64 nan 64.81 ... nan nan
    differential_phase            (site, height) float64 nan nan ... 76.05 76.4
    base_time                     (height, site) datetime64[ns] 2022-08-10T00...
    latitude                      (height, site) float64 29.62 29.62 ... 29.62
    longitude                     (height, site) float64 -95.06 ... -95.06
Attributes:
    distance_from_radar:    16.578 km
    azimuth:                6.608 degrees
    latitude_of_location:   29.62 degrees
    longitude_of_location:  -95.059 degrees

## Match the Extracted Column to the Ground Laser Disdrometer

In [ ]:
inner = [
    ["inner A"],
    ["inner B"],
]

bottom = [
    ["bottom A"],
    ["bottom B"],
]

outer_nested_mosaic = [
    ["main", inner],
    ["main", bottom],
]
axd = plt.figure(constrained_layout=True).subplot_mosaic(
    outer_nested_mosaic, empty_sentinel=None
)
identify_axes(axd, fontsize=36)

In [ ]:
mosaic = """AB
            AC"""
fig = plt.figure(constrained_layout=True)
left, right = fig.subfigures(nrows=2, ncols=1)
axd = left.subplot_mosaic(mosaic)
identify_axes(axd)

axd = right.subplot_mosaic(mosaic)
identify_axes(axd)